In [49]:
import pandas as pd
df = pd.read_csv('Terry_Stops.csv')
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 200)
import pandas_profiling
import plotly.offline as po
import plotly.graph_objs as go
%matplotlib inline

Given the information about the presence of weapons, the time of day of the call, the subject's perceived race, the officer's race

INquiry into whether race plays a role in whether or not an arrest is made


In [50]:
#Looking at the data and getting a sense of the values
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,18 - 25,-1,20150000203647,54389,Arrest,None,7412,1979,M,White,White,Male,2015-06-17T00:00:00,13:18:00,SHOPLIFT - THEFT,--THEFT - SHOPLIFT,ONVIEW,WEST PCT 2ND W - KING BEATS,N,N,East,C,C3
1,18 - 25,-1,20150000203762,54306,Field Contact,None,7725,1974,F,White,Unknown,Female,2015-06-17T00:00:00,16:36:00,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),"--DV - ARGUMENTS, DISTURBANCE (NO ARREST)",911,WEST PCT 2ND W - KING,N,N,West,K,K2
2,18 - 25,-1,20150000203886,60810,Offense Report,Lethal Cutting Instrument,6805,1973,M,White,White,Male,2015-07-06T00:00:00,17:36:00,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 2ND W - DAVID,N,Y,West,D,D1
3,18 - 25,-1,20150000203886,60811,Offense Report,Lethal Cutting Instrument,6805,1973,M,White,White,Male,2015-06-18T00:00:00,15:00:00,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 2ND W - DAVID,N,Y,West,D,D1
4,18 - 25,-1,20150000204122,54333,Arrest,Handgun,7472,1981,M,White,Black or African American,Male,2015-06-17T00:00:00,19:36:00,WARRANT - FELONY PICKUP,--WARRANT SERVICES - FELONY,ONVIEW,EAST PCT 2ND W - BEATS,N,Y,East,C,C3


In [51]:
#View the shape of the data
df.shape

(46305, 23)

In [52]:
#Identifying if there are any null values 
df.isnull().any()

Subject Age Group           False
Subject ID                  False
GO / SC Num                 False
Terry Stop ID               False
Stop Resolution             False
Weapon Type                 False
Officer ID                  False
Officer YOB                 False
Officer Gender              False
Officer Race                False
Subject Perceived Race      False
Subject Perceived Gender    False
Reported Date               False
Reported Time               False
Initial Call Type           False
Final Call Type             False
Call Type                   False
Officer Squad                True
Arrest Flag                 False
Frisk Flag                  False
Precinct                    False
Sector                      False
Beat                        False
dtype: bool

In [53]:
#Viewing the data types 
df.dtypes

Subject Age Group           object
Subject ID                   int64
GO / SC Num                  int64
Terry Stop ID                int64
Stop Resolution             object
Weapon Type                 object
Officer ID                  object
Officer YOB                  int64
Officer Gender              object
Officer Race                object
Subject Perceived Race      object
Subject Perceived Gender    object
Reported Date               object
Reported Time               object
Initial Call Type           object
Final Call Type             object
Call Type                   object
Officer Squad               object
Arrest Flag                 object
Frisk Flag                  object
Precinct                    object
Sector                      object
Beat                        object
dtype: object

In [54]:
#Started to look at value counts and first started with Precinct. We see that there are a few values like '-', 'OOJ', 'FK ERROR' and 'Unknown'. 
#We want to clean this up
df['Precinct'].value_counts()

West         11118
North        10184
-             9811
East          6109
South         5547
Southwest     2320
SouthWest      965
Unknown        200
OOJ             32
FK ERROR        19
Name: Precinct, dtype: int64

In [55]:
#Check Null Values
df.isna().sum()

Subject Age Group             0
Subject ID                    0
GO / SC Num                   0
Terry Stop ID                 0
Stop Resolution               0
Weapon Type                   0
Officer ID                    0
Officer YOB                   0
Officer Gender                0
Officer Race                  0
Subject Perceived Race        0
Subject Perceived Gender      0
Reported Date                 0
Reported Time                 0
Initial Call Type             0
Final Call Type               0
Call Type                     0
Officer Squad               605
Arrest Flag                   0
Frisk Flag                    0
Precinct                      0
Sector                        0
Beat                          0
dtype: int64

In [56]:
#Do we need this? I don't think so but want to ask Billy for deleting
#df[df['Precinct'] == 'West']

In [57]:
#We want to create our Target variable so we will check the Stop_Resolution 
df['Stop Resolution'].value_counts()

Field Contact               18555
Offense Report              15448
Arrest                      11397
Referred for Prosecution      728
Citation / Infraction         177
Name: Stop Resolution, dtype: int64

In [58]:
#We create our Target variable (Arrest)
df['Arrested'] = df['Stop Resolution'] == 'Arrest'
df['Arrested'] = df['Arrested'].replace([True, False], [1,0])

In [59]:
#I DONT THINK WE NEED THIS LINE OF CODE
#The only thing I can see us doing with this value is using it to see if we can detect a police officer who is more likely to racially profile based on on their arrest history
df['Officer ID'].value_counts()

7456      417
7634      341
7773      321
7765      315
7758      308
         ... 
5434        1
6058        1
5577        1
7624        1
6363        1
Name: Officer ID, Length: 1201, dtype: int64

In [60]:
#Changing SouthWest to Southwest to group the data together 
df['Precinct'].value_counts() 

West         11118
North        10184
-             9811
East          6109
South         5547
Southwest     2320
SouthWest      965
Unknown        200
OOJ             32
FK ERROR        19
Name: Precinct, dtype: int64

In [61]:
#IS THIS NECESSARY? PRECINCT ISN'T ONE OF OUR FEATURES THAT WE ARE MEASURING AGAINST THE ARRESTS
#Dropped OOJ, FK ERROR, and Unkown, these were a small portion of our data and we didnt think it would impact the model
df['Precinct'] = df['Precinct'].replace(['SouthWest'], ['Southwest']) 
df = df[df.Precinct != 'OOJ']
df = df[df.Precinct != 'FK ERROR']
df = df[df.Precinct != 'Unknown']
df['Precinct'].value_counts() 

West         11118
North        10184
-             9811
East          6109
South         5547
Southwest     3285
Name: Precinct, dtype: int64

In [62]:
#DO WE NEED THIS? 
df['Officer Squad'].value_counts()

TRAINING - FIELD TRAINING SQUAD                        4936
WEST PCT 1ST W - DAVID/MARY                            1521
WEST PCT 2ND W - D/M RELIEF                            1002
SOUTHWEST PCT 2ND W - FRANK                             935
NORTH PCT 2ND WATCH - NORTH BEATS                       882
WEST PCT 1ST W - KING/QUEEN                             813
WEST PCT 3RD W - DAVID                                  808
EAST PCT 3RD W - E/G RELIEF                             801
WEST PCT 1ST W - KQ/DM RELIEF                           795
EAST PCT 1ST W - E/G RELIEF (CHARLIE)                   756
EAST PCT 3RD W - EDWARD                                 736
NORTH PCT 3RD W - B/N RELIEF                            725
NORTH PCT 2ND W - NORA                                  714
WEST PCT 2ND W - KING                                   702
WEST PCT 3RD W - KING                                   684
EAST PCT 2ND W - CHARLIE RELIEF                         668
SOUTH PCT 1ST W - R/S RELIEF            

In [63]:
#We believe that an Officer's race will have an impact on our model
df['Officer Race'].value_counts()

White                            34892
Hispanic or Latino                2635
Two or More Races                 2569
Asian                             1962
Black or African American         1820
Not Specified                     1342
Nat Hawaiian/Oth Pac Islander      447
American Indian/Alaska Native      317
Unknown                             70
Name: Officer Race, dtype: int64

In [64]:
#We want to group the 'Unkown' with 'Not Specified'
df['Officer Race'] = df['Officer Race'].replace(['Unknown'], ['Not Specified'])

In [65]:
df['Officer Race'].value_counts()

White                            34892
Hispanic or Latino                2635
Two or More Races                 2569
Asian                             1962
Black or African American         1820
Not Specified                     1412
Nat Hawaiian/Oth Pac Islander      447
American Indian/Alaska Native      317
Name: Officer Race, dtype: int64

In [66]:
#Creating the value of whether or not the officer is White or not and look at the value counts to see what that data looks like 
df['White_Officer'] = (df['Officer Race'] == 'White')
df['White_Officer'] = df['White_Officer'].replace([True, False], [1,0])
df['White_Officer'].value_counts()

1    34892
0    11162
Name: White_Officer, dtype: int64

In [67]:
#We also think that a Subject's Perceived Race will have an impact on the rate of arrests so we look at those value counts 
df['Subject Perceived Race'].value_counts()

White                                        22497
Black or African American                    13719
Unknown                                       2506
-                                             1863
Hispanic                                      1664
Asian                                         1479
American Indian or Alaska Native              1326
Multi-Racial                                   800
Other                                          152
Native Hawaiian or Other Pacific Islander       48
Name: Subject Perceived Race, dtype: int64

In [68]:
#We converted '-','Unknown' into 'Not Specified', we left 'Other' in its own category
df['Subject Perceived Race'] = df['Subject Perceived Race'].replace(['-', 'Unknown'],['Not Specified', 'Not Specified'])
df['Subject Perceived Race'].value_counts()

White                                        22497
Black or African American                    13719
Not Specified                                 4369
Hispanic                                      1664
Asian                                         1479
American Indian or Alaska Native              1326
Multi-Racial                                   800
Other                                          152
Native Hawaiian or Other Pacific Islander       48
Name: Subject Perceived Race, dtype: int64

In [69]:
#We want to see if there is a relationship of arrests and race so we created a feature called 'Subject_POC' by grouping all those who do not have 'White' as their value
df['Subject_POC'] = df['Subject Perceived Race'] != 'White'
df['Subject_POC'] = df['Subject_POC'].replace([True, False], [1,0])
df['Subject_POC'].value_counts()

1    23557
0    22497
Name: Subject_POC, dtype: int64

In [70]:
#We want to explore percieved gender
df['Subject Perceived Gender'].value_counts()

Male                                                         36088
Female                                                        9354
Unable to Determine                                            314
-                                                              272
Unknown                                                         22
Gender Diverse (gender non-conforming and/or transgender)        4
Name: Subject Perceived Gender, dtype: int64

In [71]:
#Grouping Unable to Determine with Gender Diverse. We recognize that although they may not identify as Gender Diverse we know that perception of one's gender is an indication of how 
#Society treats you. This in no way is being used to disregard individuals' gender identity.
df['Subject Perceived Gender'] = df['Subject Perceived Gender'].replace('-', 'Unknown')
df['Subject Perceived Gender'] = df['Subject Perceived Gender'].replace('Unable to Determine', 'Gender Diverse (gender non-conforming and/or transgender)')
df['Subject Perceived Gender'].value_counts()

Male                                                         36088
Female                                                        9354
Gender Diverse (gender non-conforming and/or transgender)      318
Unknown                                                        294
Name: Subject Perceived Gender, dtype: int64

In [72]:
#Look at the initial call type and the value counts 
df['Initial Call Type'].value_counts()

-                                                     13056
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW             3093
SUSPICIOUS PERSON, VEHICLE OR INCIDENT                 2905
DISTURBANCE, MISCELLANEOUS/OTHER                       2378
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)         1945
TRESPASS                                               1855
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)              1374
SHOPLIFT - THEFT                                       1315
FIGHT - IP - PHYSICAL (NO WEAPONS)                     1253
WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)         1123
ROBBERY - IP/JO (INCLUDES STRONG ARM)                   878
BURG - IP/JO - RES (INCL UNOCC STRUCTURES)              766
THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING)           711
DIST - IP/JO - DV DIST - NO ASLT                        686
NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)        686
PROPERTY - DAMAGE                                       675
ASLT - IP/JO - DV                       

In [73]:
#Look at the Final Call type
#WE MAY WANT TO CREATE A FEATURE CALLED VIOLENT AND NON VIOLENT. 
df['Final Call Type'].value_counts()

-                                           13056
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON     3650
--PROWLER - TRESPASS                         3272
--DISTURBANCE - OTHER                        2653
--ASSAULTS, OTHER                            2237
                                            ...  
-OFF DUTY EMPLOYMENT                            1
WARRANT PICKUP - FROM OTHER AGENCY              1
RAPE                                            1
SHOTS -DELAY/INCLUDES HEARD/NO ASSAULT          1
ESCAPE - PRISONER                               1
Name: Final Call Type, Length: 204, dtype: int64

In [74]:
#Looking at Call Type count
df['Call Type'].value_counts()

911                              20666
-                                13056
ONVIEW                            8816
TELEPHONE OTHER, NOT 911          3194
ALARM CALL (NOT POLICE ALARM)      315
TEXT MESSAGE                         7
Name: Call Type, dtype: int64

In [75]:
#Turning the Year_of_Report to datetime value 
df['Year_of_Report'] = pd.to_datetime(df['Reported Date']).dt.year

In [76]:
#Turning the Officer YOB into an interger
df['Officer YOB'] = df['Officer YOB'].astype(int)

In [77]:
df['Year_of_Report'] = df['Year_of_Report'].astype(int)

In [79]:
df.dtypes

Subject Age Group           object
Subject ID                   int64
GO / SC Num                  int64
Terry Stop ID                int64
Stop Resolution             object
Weapon Type                 object
Officer ID                  object
Officer YOB                  int64
Officer Gender              object
Officer Race                object
Subject Perceived Race      object
Subject Perceived Gender    object
Reported Date               object
Reported Time               object
Initial Call Type           object
Final Call Type             object
Call Type                   object
Officer Squad               object
Arrest Flag                 object
Frisk Flag                  object
Precinct                    object
Sector                      object
Beat                        object
Arrested                     int64
White_Officer                int64
Subject_POC                  int64
Year_of_Report               int64
dtype: object

In [80]:
df['Officer_Age'] = df['Year_of_Report'] - df['Officer YOB']

In [81]:
df

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat,Arrested,White_Officer,Subject_POC,Year_of_Report,Officer_Age
0,18 - 25,-1,20150000203647,54389,Arrest,None,7412,1979,M,White,White,Male,2015-06-17T00:00:00,13:18:00,SHOPLIFT - THEFT,--THEFT - SHOPLIFT,ONVIEW,WEST PCT 2ND W - KING BEATS,N,N,East,C,C3,1,1,0,2015,36
1,18 - 25,-1,20150000203762,54306,Field Contact,None,7725,1974,F,White,Not Specified,Female,2015-06-17T00:00:00,16:36:00,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),"--DV - ARGUMENTS, DISTURBANCE (NO ARREST)",911,WEST PCT 2ND W - KING,N,N,West,K,K2,0,1,1,2015,41
2,18 - 25,-1,20150000203886,60810,Offense Report,Lethal Cutting Instrument,6805,1973,M,White,White,Male,2015-07-06T00:00:00,17:36:00,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 2ND W - DAVID,N,Y,West,D,D1,0,1,0,2015,42
3,18 - 25,-1,20150000203886,60811,Offense Report,Lethal Cutting Instrument,6805,1973,M,White,White,Male,2015-06-18T00:00:00,15:00:00,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,ONVIEW,WEST PCT 2ND W - DAVID,N,Y,West,D,D1,0,1,0,2015,42
4,18 - 25,-1,20150000204122,54333,Arrest,Handgun,7472,1981,M,White,Black or African American,Male,2015-06-17T00:00:00,19:36:00,WARRANT - FELONY PICKUP,--WARRANT SERVICES - FELONY,ONVIEW,EAST PCT 2ND W - BEATS,N,Y,East,C,C3,1,1,1,2015,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46300,18 - 25,-1,20150000202641,53780,Field Contact,None,6761,1971,M,Hispanic or Latino,Black or African American,Male,2015-06-16T00:00:00,18:07:00,ROBBERY - IP/JO (INCLUDES STRONG ARM),--ROBBERY - STRONG ARM,911,SOUTH PCT 1ST W - ROBERT,N,N,South,R,R3,0,0,1,2015,44
46301,18 - 25,-1,20150000202641,53781,Field Contact,None,6761,1971,M,Hispanic or Latino,Black or African American,Male,2015-06-16T00:00:00,18:23:00,ROBBERY - IP/JO (INCLUDES STRONG ARM),--ROBBERY - STRONG ARM,911,SOUTH PCT 1ST W - ROBERT,N,N,South,R,R3,0,0,1,2015,44
46302,18 - 25,-1,20150000202820,55102,Offense Report,None,7590,1968,M,Hispanic or Latino,Hispanic,Male,2015-06-16T00:00:00,20:52:00,FIGHT - IP - PHYSICAL (NO WEAPONS),--DISTURBANCE - FIGHT,911,SOUTHWEST PCT 2ND W - FRANK,N,N,Southwest,F,F3,0,0,1,2015,47
46303,18 - 25,-1,20150000203317,54271,Offense Report,None,7792,1987,M,White,Black or African American,Female,2015-06-17T00:00:00,10:52:00,FIGHT - IP - PHYSICAL (NO WEAPONS),"--ASSAULTS, OTHER",911,TRAINING - FIELD TRAINING SQUAD,N,N,West,M,M1,0,1,1,2015,28


In [34]:
df['Hour'] = pd.to_datetime(df['Reported Time'], format='%H:%M:%S').dt.hour

In [35]:
df['Hour'] = df['Hour'].replace(0,24)

In [36]:
df['Morning'] = (df['Hour'] >= 5) & (df['Hour'] < 12)
df['Morning'] = df['Morning'].replace([True,False], [1,0])

In [37]:
df['Afternoon']= (df['Hour'] >= 12) & (df['Hour'] < 17)
df['Afternoon'] = df['Afternoon'].replace([True,False], [1,0])

In [38]:
df['Evening'] = (df['Hour'] >= 17) & (df['Hour'] < 21)
df['Evening'] = df['Evening'].replace([True,False], [1,0])

In [39]:
df['Night'] = (df['Hour'] >= 21) & (df['Hour'] <= 24)
df['Night'] = df['Night'].replace([True,False], [1,0])

In [40]:
df['Hour'].value_counts()

18    2809
2     2692
19    2534
3     2459
17    2434
23    2287
1     2286
16    2212
15    2119
24    2095
14    2026
22    1949
11    1888
13    1802
10    1788
21    1645
12    1569
5     1467
9     1426
4     1423
20    1413
8     1277
7     1266
6     1188
Name: Hour, dtype: int64

In [44]:
df['Officer YOB'].value_counts()

1986    3229
1987    2946
1984    2709
1991    2647
1985    2473
1992    2376
1990    2185
1988    2045
1989    1948
1982    1833
1983    1682
1979    1485
1993    1412
1981    1392
1971    1213
1978    1139
1995    1099
1976     991
1977     986
1973     912
1994     886
1980     794
1967     706
1968     616
1996     596
1970     576
1974     552
1969     534
1975     519
1962     452
1972     417
1964     413
1965     411
1997     373
1963     258
1966     222
1958     214
1961     211
1959     173
1960     160
1900      70
1954      44
1957      43
1953      32
1955      21
1956      17
1998      14
1948      11
1952       9
1949       5
1946       2
1951       1
Name: Officer YOB, dtype: int64

In [82]:
#Created a value of 'under_17' from the 'Subject Age Group'
df['under_17'] = df['Subject Age Group']=='1 - 17'
df['under_17'] = df['under_17'].replace([True, False], [1,0])

In [ ]:
features = ['Subject Age Group', 'Officer Race', 'Subject Perceived Race', 'Officer Gender', 'Stop Resolution', 'Subject Perceived Gender', 
            'White Officer', 'POC', 'Officer_Age', 'under_17','Morning', 'Afternoon', 'Evening', 'Night']


In [ ]:
len(features)

In [ ]:
target = df['Arrested']

In [ ]:
df_features = df[features]

In [ ]:
pandas_profiling.ProfileReport(df_features)

In [ ]:
df_features.drop(columns = 'Arrest Flag', inplace = True)